In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv

In [3]:
N = 23 # Numero de periodos
mYah = 3 # Numero de acciones extraidas de Yahoo
mFin = 4 # Numero de acciones extraidas de Finance.com
m = mYah + mFin # Numero de acciones

precios = np.zeros((N+1, m))
retornos = np.zeros((N, m))

Extrayendo columnas de los archivos de Yahoo Finance:

In [4]:
archivosYahoo = ['csvs/AVH.csv', 'csvs/CIB.csv', 'csvs/EC.csv']

In [5]:
for k in range(mYah):
    archivo = archivosYahoo[k]
    lector = csv.reader(open(archivo))
    lector.__next__() #Ignorar primer renglon
    
    for i in range(N+1):
        precios[i, k] = lector.__next__()[5] #Extraer la 5ta columna: precio de cierre ajustado
        if (i > 0):
            retornos[i-1, k] = (precios[i, k] - precios[i-1, k])/precios[i-1, k]

Extrayendo columnas de los archivos de Investing.com

In [13]:
 archivosFin = ['csvs/ARG-2.csv', 'csvs/CFV-2.csv', 'csvs/FTSE-2.csv', 'csvs/IMI-2.csv']

In [15]:
for k in range(mYah, mYah + mFin):
    archivo = archivosFin[k - mYah]
    lector = csv.reader(open(archivo))
    lector.__next__() #Ignorar primer renglon
    
    for i in range(N, -1, -1):
        precios[i, k] = lector.__next__()[1] #Extraer la 1era columna: precio de cierre
        if (i < N):
            retornos[i, k] = (precios[i+1, k] - precios[i, k])/precios[i, k]

In [16]:
print(retornos)

[[ 0.1045577   0.0227815   0.04598737  0.03225806  0.01818188  0.00953783
   0.00244798]
 [ 0.04285705  0.02719505  0.26120685  0.01953125  0.0027462   0.06342655
   0.01098901]
 [ 0.01868006  0.1677591   0.28981542 -0.01149425 -0.07945268  0.05065252
   0.07971014]
 [ 0.14303185 -0.08204569 -0.07207213 -0.06395349 -0.09821422 -0.06187936
  -0.06152125]
 [-0.0941177  -0.00379325  0.10394077 -0.02587992 -0.01815318 -0.01995787
  -0.01787843]
 [-0.0354192   0.1441194   0.14226573  0.06269926  0.01512874  0.10005898
   0.02912621]
 [-0.07711124 -0.02685693  0.00224612 -0.029       0.04056195 -0.00841931
  -0.02712264]
 [-0.1432361   0.03018537 -0.0437413   0.02265705  0.00954689  0.02047929
  -0.01818182]
 [ 0.00928784 -0.03174548  0.03990269 -0.02316213 -0.0618411  -0.03754528
   0.00246914]
 [-0.14465983 -0.05554349  0.05755737 -0.01546392  0.01767677  0.01847702
  -0.02339901]
 [ 0.0405855  -0.03782289  0.19159285 -0.14659686 -0.09015715  0.00861483
  -0.08070618]
 [-0.09852444 -0.1074